## 2. Features
Looking at the EDA we there are several data cleaning items as well as features that we can build

Drop Columns:
- Z_CostContact
- Z_Revenue

Fix items:
- drop ages over 100
- impute income
- Marital status

Features:
- create age
- see how long the customer has been a client
- create total children


In [72]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import datetime

In [65]:
data = pd.read_csv(f'marketing_campaign.csv', sep=';')

In [73]:
class FeatureEngineering:
    def __init__(self, data, preprocessing_type='test'):
        self.data = data
        self.median_income = None
        self.income_cutoff = None
        self.preprocessing_type = preprocessing_type

    def preprocessing(self):
        # Calculate Age and remove outliers
        self.data['Age'] = datetime.datetime.now().year - self.data['Year_Birth'] 
        self.data = self.data[self.data['Age'] < 100]

        # Calculate Tenure
        self.data['Dt_Customer'] = pd.to_datetime(self.data['Dt_Customer'])
        self.data['Tenure'] = datetime.datetime.now().year - self.data['Dt_Customer'].dt.year

        # Create Total Children
        self.data['Total_Children'] = self.data['Kidhome'] + self.data['Teenhome']

        # Fix Marital Status
        self.data['Marital_Status'] = self.data['Marital_Status'].replace(['YOLO', 'Alone', 'Absurd'], 'Single')

        # Remove outlier and impute median income
        if self.preprocessing_type == 'train':
            Q3 = np.quantile(data['Income'].dropna(), 0.75)
            Q1 = np.quantile(data['Income'].dropna(), 0.25)
            IQR = Q3 - Q1
            self.income_cutoff = Q3 + 1.5 * IQR
            self.median_income = self.data.loc[self.data['Income'] < self.income_cutoff, 'Income'].median()

        self.data.loc[self.data['Income'] > self.income_cutoff, 'Income'] = self.median_income
        self.data['Income'] = self.data['Income'].fillna(self.median_income)

        # Set ID as index
        self.data = self.data.set_index('ID')

        # Drop columns
        self.data = self.data.drop(['Year_Birth', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue'], axis=1)
    
    def get_data(self) -> pd.DataFrame:
        return(self.data)

In [74]:
processing_train = FeatureEngineering(data, preprocessing_type='train')
processing_train.preprocessing()
preprocessed_data = processing_train.get_data()

In [77]:
preprocessed_data.head()

,Education,Marital_Status,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Tenure,Total_Children
ID,,,,,,,,,,,,,,,,,,,,,
5524,Graduation,Single,58138.0,0,0,58,635,88,546,172,...,0,0,0,0,0,0,1,66,11,0
2174,Graduation,Single,46344.0,1,1,38,11,1,6,2,...,0,0,0,0,0,0,0,69,9,2
4141,Graduation,Together,71613.0,0,0,26,426,49,127,111,...,0,0,0,0,0,0,0,58,10,0
6182,Graduation,Together,26646.0,1,0,26,11,4,20,10,...,0,0,0,0,0,0,0,39,9,1
5324,PhD,Married,58293.0,1,0,94,173,43,118,46,...,0,0,0,0,0,0,0,42,9,1


In [80]:
preprocessed_data.describe()

,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,...,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response,Age,Tenure,Total_Children
count,2205.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,...,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000,2237.000000
mean,51622.094785,0.444345,0.506482,49.104604,303.995530,26.270451,166.916853,37.523022,27.068842,43.968708,...,0.072865,0.074654,0.072418,0.064372,0.013411,0.008941,0.149307,54.098346,9.972284,0.950827
std,20713.063826,0.538467,0.544593,28.956073,336.574382,39.715972,225.661158,54.639909,41.293949,52.054318,...,0.259974,0.262890,0.259237,0.245469,0.115052,0.094152,0.356471,11.701917,0.684704,0.752037
min,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,27.000000,9.000000,0.000000
25%,35196.000000,0.000000,0.000000,24.000000,24.000000,1.000000,16.000000,3.000000,1.000000,9.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,46.000000,10.000000,0.000000
50%,51287.000000,0.000000,0.000000,49.000000,174.000000,8.000000,67.000000,12.000000,8.000000,24.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53.000000,10.000000,1.000000
75%,68281.000000,1.000000,1.000000,74.000000,504.000000,33.000000,232.000000,50.000000,33.000000,56.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,64.000000,10.000000,1.000000
max,113734.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,263.000000,362.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,83.000000,11.000000,3.000000


In [79]:
# preprocessing_test = FeatureEngineering(data, preprocessing_type='test')
# preprocessing_test.median_income = processing_train.median_income
# preprocessing_test.income_cutoff = processing_train.income_cutoff
# preprocessing_test.preprocessing()
# preprocessing_test.get_data()